In [ ]:
# GFS: https://rda.ucar.edu/datasets/ds084.1/

In [ ]:
import sys, os
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def check_file_status(filepath, filesize):
    sys.stdout.write('\r')
    sys.stdout.flush()
    size = int(os.stat(filepath).st_size)
    percent_complete = (size/filesize)*100
    sys.stdout.write('%.3f %s' % (percent_complete, '% Completed'))
    sys.stdout.flush()

In [ ]:
# Try to get password
if len(sys.argv) < 4 and not 'RDAPSWD' in os.environ:
    try:
        import getpass
        input = getpass.getpass
    except:
        try:
            input = raw_input
        except:
            pass
    pswd = input('Password: ')
else:
    try:
        pswd = sys.argv[1]
    except:
        pswd = os.environ['RDAPSWD']

In [ ]:
url = 'https://rda.ucar.edu/cgi-bin/login'

values = {'email' : 'm.wessler@utah.edu', 'passwd' : pswd, 'action' : 'login'}

# Authenticate
ret = requests.post(url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)
    
dspath = 'https://rda.ucar.edu/data/ds084.1/'

In [ ]:
init_dates = pd.date_range(datetime(2015, 1, 15, 0, 0), 
              #datetime(2020, 5, 31, 0, 0), 
              datetime(2015, 1, 16, 0, 0), 
              freq='12H')

forecast_hours = np.arange(0, 180+1, 3)

filelist = []
for init in init_dates:
    
        filelist.append([('{0:04d}/{0:04d}{1:02d}{2:02d}/' + 
            'gfs.0p25.{0:04d}{1:02d}{2:02d}{3:02d}.' + 
            'f{4:03d}.grib2').format(
            init.year, init.month, init.day, init.hour, fhr)
                    for fhr in forecast_hours])

In [ ]:
for flist in filelist:
    
    for file in flist:

        filename=dspath+file
        file_base = os.path.basename(file)

        print('Downloading',file_base)

        req = requests.get(filename, cookies = ret.cookies, allow_redirects=True, stream=True)
        filesize = int(req.headers['Content-length'])

        with open(file_base, 'wb') as outfile:
            chunk_size=1048576
            for chunk in req.iter_content(chunk_size=chunk_size):
                outfile.write(chunk)
                if chunk_size < filesize:
                    check_file_status(file_base, filesize)

        check_file_status(file_base, filesize)
        print()